In [12]:
import pandas

In [1]:
from pycrucible import CrucibleClient
import os
from dotenv import load_dotenv
load_dotenv()
crucible_api_key = os.getenv("CRUCIBLE_API_KEY")
crucible_client = CrucibleClient(
    api_url="https://crucible.lbl.gov/testapi",
    api_key=crucible_api_key # v3
)


In [2]:
%reload_ext crucible_project_graph
import crucible_project_graph as projgraph


In [3]:
projgraph.generate_project_cache('10k_perovskites',crucible_client, include_metadata=True)
proj = projgraph.get_project('10k_perovskites',crucible_client)

getting samples and datasets from Crucible


In [17]:
G = projgraph.generate_project_graph('10k_Perovskites',crucible_client) 
G = projgraph.load_sample_graph('10k_Perovskites') 

['', '0ta8e49e4hyev00048txb5esvg', '0ta37t3xvhv7q000097zkng08w', '0ta8e4gc99v2n000s2z8hyjvdw', '0ta37t5fass45000nv44qg2qtw', '0ta3eg18p5vmz000a1vy3wdgew', '0tb5g0n3dhx23001mkaw68g2mr', '0ta3er35bhtsb000ky1zvwh2qr', '0ta3etcc81z6b000jbekfxt58m', '0ta3etydd1sjn000f9k9vnydcg', '0tb5j4fmcnzrk0013zt4yrxvac', '0ta32qpr41zgx000qqkqad4sa0', '0ta32qw3sstyn000wcks3b1djg', '0ta32sdwynty50004jdzb9bcec', '0ta32tdnv9vy50004s7d2nbjh4', '0ta32thazsv0n0008t8emmqby8', '0ta32tjpn5xr300041t7f8r2g0', '0ta32tr2t1y5s000sgq1fcf7br', '0ta3795fwswy1000jasq7gqc74', '0ta3797yc5xr7000xsw26ah244', '0ta3798y99r8x00095q5cb0ebr', '0ta3799z0xz07000df6p559dcw', '0ta379ayv5rjx000wtcxbfa2s0', '0ta379byqdr97000ptfe7b8hzr', '0t9ssr8mp5yd3000e0q2rprcxw', '0ta37tec3dscx00076ssbr6mg0', '0tb5g0n3e1xtf001y3e9b47jvg', '0tb5j4fmcnzrk000e2sphnhntw', '0ta32qry1xzg1000kbddc9cq8m', '0ta32qtq5hw85000emw2ae5x8m', '0ta32sgbahr1v000w46hbtmte4', '0ta32shze1rn3000pj23nmhwbc', '0ta32skcq1xhx000dv49xtwprm', '0ta32tfvg5ye50000c6d1kgc1c', '0ta3

In [6]:
import networkx as nx
#nx.draw_networkx(G, with_labels=False)

In [7]:
s = proj['samples_by_name']['TF000275']
s

{'id': 1368,
 'unique_id': '0tbawsqfchttn00013xwxpb4zc',
 'sample_name': 'TF000275',
 'owner_orcid': '0000-0002-8792-9669',
 'date_created': '2026-01-16T17:31:26.963000-08:00',
 'description': '',
 'datasets': [{'id': 121103,
   'unique_id': '0tbawtccw1tav000955khhgxnw',
   'dataset_name': 'Spin Run for TF000275 - 0tbawsqfc1t17',
   'measurement': 'spin_run'},
  {'id': 121133,
   'unique_id': '0tbaz40bbnzz1000p019xj9rxw',
   'dataset_name': '260116_185040_pollux_oospec_multipos_line_scan',
   'measurement': 'pollux_oospec_multipos_line_scan'}]}

In [8]:
# immediate parents
[proj['samples_by_id'][sid] for sid in G.predecessors(s['unique_id'])]
# immediate children
[proj['samples_by_id'][sid] for sid in G.successors(s['unique_id'])]
# all ancestors
ancestors = nx.ancestors(G, s['unique_id'])
ancestors = [proj['samples_by_id'][sid] for sid in ancestors]
# all descendants
descendants = nx.descendants(G, s['unique_id'])
descendants = [proj['samples_by_id'][sid] for sid in descendants]

In [9]:
ancestors

[{'id': 1231,
  'unique_id': '0tb8ckbgc9xex000cngfgm41dg',
  'sample_name': 'SS00014',
  'owner_orcid': '0000-0002-6381-3119',
  'date_created': '2026-01-14T19:53:58.933855-08:00',
  'description': 'NaN',
  'datasets': [{'id': 120765,
    'unique_id': '0tb8ckbtg9v710002hjr0aaw0c',
    'dataset_name': 'Stock Solution recipe for SS00014',
    'measurement': 'Stock Solution synthesis'}]},
 {'id': 1244,
  'unique_id': '0tb8cmcj9dvf1000m4xnfd33r4',
  'sample_name': 'PS000039',
  'owner_orcid': '0000-0002-6381-3119',
  'date_created': '2026-01-14T19:55:05.054765-08:00',
  'description': 'NaN',
  'datasets': [{'id': 120778,
    'unique_id': '0tb8cmcyxss8k000bed1q17rrg',
    'dataset_name': 'Precursor Solution recipe for PS000039',
    'measurement': 'Precursor Solution synthesis'}]},
 {'id': 1366,
  'unique_id': '0tbawsqfc1t17001ms29pfxtp0',
  'sample_name': 'TRAY00035',
  'owner_orcid': '0000-0002-8792-9669',
  'date_created': '2026-01-16T17:31:26.963000-08:00',
  'description': None,
  'dat

In [10]:
# Find the solid precursor samples that were used to make this thin film sample
solid_precursors = [sample for sample in ancestors if sample['sample_name'].startswith('SP')]
solid_precursors
# Capture their compositions from the 'Solid Precursor synthesis' dataset
precursor_compositions = []
for sp in solid_precursors:
    for ds in sp['datasets']:
        if ds['measurement'] == 'Solid Precursor synthesis':
            full_ds = proj['datasets_by_id'][ds['unique_id']]
            print(full_ds['dataset_name'])
            material = full_ds['scientific_metadata']['name']
            precursor_compositions.append((sp['sample_name'], material))
precursor_compositions


Solid Precursor recipe for SP000016
Solid Precursor recipe for SP000013


[('SP000016', '3-(tert-butoxy)-3-oxopropan-1-aminium iodide'),
 ('SP000013', 'bismuth(iii) iodide')]

In [11]:
thin_films = [s for s in proj['samples'] 
                if s['sample_name'].startswith('TF')]
thin_films.sort(key= lambda x: x['sample_name'])


rows = []

for s in thin_films:
    # all ancestors
    ancestors = nx.ancestors(G, s['unique_id'])
    ancestors = [proj['samples_by_id'][sid] for sid in ancestors]
    # all descendants
    descendants = nx.descendants(G, s['unique_id'])
    descendants = [proj['samples_by_id'][sid] for sid in descendants]

    # Find the solid precursor samples that were used to make this thin film sample
    solid_precursors = [sample for sample in ancestors if sample['sample_name'].startswith('SP')]
    solid_precursors
    # Capture their compositions from the 'Solid Precursor synthesis' dataset
    precursor_compositions = []
    for sp in solid_precursors:
        for ds in sp['datasets']:
            if ds['measurement'] == 'Solid Precursor synthesis':
                full_ds = proj['datasets_by_id'][ds['unique_id']]
                material = full_ds['scientific_metadata']['name']
                precursor_compositions.append(material)
    
    if len(precursor_compositions) != 2:
        #print(f"Warning: expected 2 solid precursors for {s['sample_name']}, found {len(precursor_compositions)}")
        if len(precursor_compositions) < 2:
            precursor_compositions += [None] * (2 - len(precursor_compositions))

    row = {
        'thin_film_sample_name': s['sample_name'],
        'thin_film_unique_id': s['unique_id'],
        'sp_A': precursor_compositions[0],
        'sp_B': precursor_compositions[1],
    }
    #print(row)
    rows.append(row)    

df = pandas.DataFrame(rows)
df

,thin_film_sample_name,thin_film_unique_id,sp_A,sp_B
0,TF000001,0ta8bqjytdv6b000px06bejczm,bismuth (iii) iodide,tyrammonium iodide
1,TF000002,0ta8d1axm9s7v0009yx9mzsrmc,bismuth (iii) iodide,tyrammonium iodide
2,TF000003,0ta8dg128nwdn000r53vcwedbg,bismuth (iii) iodide,tyrammonium iodide
3,TF000004,0ta8dja1fns01000a0z0n9krar,bismuth (iii) iodide,tyrammonium iodide
4,TF000005,0ta8dk60s5xgz000er584gpve0,bismuth (iii) bromide,tyrammonium bromide
...,...,...,...,...
571,TF000572,0tbhf9m65hveq00492ndmj8eg4,None,None
572,TF000573,0tbhf9m65hveq005rycn67dbsw,None,None
573,TF000574,0tbhf9m65hveq0062v889rk8pg,None,None
574,TF000575,0tbhf9m65hveq007c2fczkzm6g,None,None


In [19]:
# find datasets that are "Solid Precursor synthesis" and flatten
rows = []

sp_ds = []
for ds in proj['datasets']:
    if ds['measurement'] == 'Solid Precursor synthesis':
        full_ds = proj['datasets_by_id'][ds['unique_id']]
        sp_ds.append(full_ds)

pandas.json_normalize(sp_ds)


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.sample_name,scientific_metadata.sample_description,scientific_metadata.notes,scientific_metadata.cas,scientific_metadata.rfid,scientific_metadata.name,scientific_metadata.abbrev,scientific_metadata.vendor,scientific_metadata.opened_timestamp,scientific_metadata.storage_location
0,10k_perovskites,None,None,None,118892,None,None,False,None,Solid Precursor recipe for SP000001,...,SP000001,None,NaN,NaN,NaN,tyrammonium iodide,PEAOHI,Homemade,2025-10-20,6207 Bench
1,10k_perovskites,None,None,None,118893,None,None,False,None,Solid Precursor recipe for SP000002,...,SP000002,None,NaN,NaN,NaN,tyrammonium bromide,PEAOHBr,Homemade,2024-07-04,6207 Bench
2,10k_perovskites,None,None,None,118894,None,None,False,None,Solid Precursor recipe for SP000003,...,SP000003,None,NaN,7787-64-6,C0084566,bismuth (iii) iodide,BiI3,Sigma,2024-10-08,Glovebox
3,10k_perovskites,None,None,None,118895,None,None,False,None,Solid Precursor recipe for SP000004,...,SP000004,None,NaN,7787-58-8,C0169369,bismuth (iii) bromide,BiBr3,Sigma,2024-12-18,Glovebox
4,10k_perovskites,None,None,None,118910,None,None,False,None,Solid Precursor recipe for test3,...,test3,None,NaN,None,None,None,None,None,None,None
5,10k_perovskites,None,None,None,118911,None,None,False,None,Solid Precursor recipe for test5,...,test5,None,NaN,None,None,None,None,None,None,None
6,10k_perovskites,None,None,None,118928,None,None,False,None,Solid Precursor recipe for SP000005,...,SP000005,None,NaN,7787-64-6,C0084566,bismuth (iii) iodide,BiI3,Sigma,2024-10-08,Nano Glovebox
7,10k_perovskites,None,None,None,118929,None,None,False,None,Solid Precursor recipe for SP000006,...,SP000006,None,NaN,7787-58-8,C0169369,bismuth (iii) bromide,BiBr3,Sigma,2024-12-18,Nano Glovebox
8,10k_perovskites,None,None,None,118930,None,None,False,None,Solid Precursor recipe for SP000007,...,SP000007,None,NaN,NaN,NaN,tyrammonium iodide,PEAOHI,Homemade,2025-10-20,6207 Bench
9,10k_perovskites,None,None,None,118931,None,None,False,None,Solid Precursor recipe for SP000008,...,SP000008,None,NaN,NaN,NaN,3-(tert-butoxy)-3-oxopropan-1-aminium iodide,tBuO2CEAI,Homemade,None,6207 Bench


In [21]:
# apply to all dataset types in project
measurement_types = set([ds['measurement'] for ds in proj['datasets']])
display(measurement_types)

measurement_dfs = dict()

for measurement in measurement_types:
    measurement_ds = []
    for ds in proj['datasets']:
        if ds['measurement'] == measurement:
            full_ds = proj['datasets_by_id'][ds['unique_id']]
            measurement_ds.append(full_ds)

    measurement_dfs[measurement] = pandas.json_normalize(measurement_ds)

    display(measurement_dfs[measurement])



{'Precursor Solution synthesis',
 'Solid Precursor synthesis',
 'Stock Solution synthesis',
 'pollux_oospec_multipos_line_scan',
 'sample well image TEST',
 'spin_run',
 'thin film carrier image'}

,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_014_TF000559.y_positions,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.integration_time,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.sample_name,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.sample_uuid,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.tray_name,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.tray_uuid,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.x_center,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.x_positions,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.y_center,scientific_metadata.measurement.pollux_oospec_multipos_line_scan.positions.pos_015_TF000560.y_positions
0,10k_perovskites,None,51,None,119825,C:/Users/lab/Documents/nirvana/data,a785cb7557c430bfff7575a9d4f1aed9ce0b9a53c15b9b...,False,nirvana spectrometer,251210_173817_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10k_perovskites,None,51,None,119826,C:/Users/lab/Documents/nirvana/data,f3b6b3f54b075efeff7b04beef1409a7ecf50a33425b59...,False,nirvana spectrometer,251218_130904_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10k_perovskites,None,51,None,119827,C:/Users/lab/Documents/nirvana/data,e01cfbbf07bc077fff6b5013728e2ea0214aa2349d1447...,False,nirvana spectrometer,251218_133226_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10k_perovskites,None,51,None,119828,C:/Users/lab/Documents/nirvana/data,c144744b7baeef6a66cb9cd9121d2b5fb385d3c697c6e3...,False,nirvana spectrometer,251218_131602_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10k_perovskites,None,51,None,119829,C:/Users/lab/Documents/nirvana/data,0c522d8b060e8fe19d09948083cf0b9dd71fd4dd9f7456...,False,nirvana spectrometer,251218_131902_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10k_perovskites,None,51,None,119830,C:/Users/lab/Documents/nirvana/data,167c0384bd2d16bd43444e2b01fab990f14b0dd3559ca9...,False,nirvana spectrometer,251218_132216_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10k_perovskites,None,51,None,119831,C:/Users/lab/Documents/nirvana/data,58bb95ed538f913a1bf86553eba5578ea698787e42e493...,False,nirvana spectrometer,251218_132519_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10k_perovskites,None,51,None,119832,C:/Users/lab/Documents/nirvana/data,8a89c8c7481ee848d34797ff935bc3a552c27dc095d299...,False,nirvana spectrometer,251218_132822_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10k_perovskites,None,51,None,119833,C:/Users/lab/Documents/nirvana/data,fddb15e11d1515050d6984bf3bd2cba88ab1fa0cb3ee71...,False,nirvana spectrometer,251218_182249_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10k_perovskites,None,51,None,119834,C:/Users/lab/Documents/nirvana/data,b73d0f5de2e70096b06c91658e0659d6e71e30579f8e43...,False,nirvana spectrometer,251218_185803_pollux_oospec_multipos_line_scan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.sample_name,scientific_metadata.sample_description,scientific_metadata.notes,scientific_metadata.cas,scientific_metadata.rfid,scientific_metadata.name,scientific_metadata.abbrev,scientific_metadata.vendor,scientific_metadata.opened_timestamp,scientific_metadata.storage_location
0,10k_perovskites,None,None,None,118892,None,None,False,None,Solid Precursor recipe for SP000001,...,SP000001,None,NaN,NaN,NaN,tyrammonium iodide,PEAOHI,Homemade,2025-10-20,6207 Bench
1,10k_perovskites,None,None,None,118893,None,None,False,None,Solid Precursor recipe for SP000002,...,SP000002,None,NaN,NaN,NaN,tyrammonium bromide,PEAOHBr,Homemade,2024-07-04,6207 Bench
2,10k_perovskites,None,None,None,118894,None,None,False,None,Solid Precursor recipe for SP000003,...,SP000003,None,NaN,7787-64-6,C0084566,bismuth (iii) iodide,BiI3,Sigma,2024-10-08,Glovebox
3,10k_perovskites,None,None,None,118895,None,None,False,None,Solid Precursor recipe for SP000004,...,SP000004,None,NaN,7787-58-8,C0169369,bismuth (iii) bromide,BiBr3,Sigma,2024-12-18,Glovebox
4,10k_perovskites,None,None,None,118910,None,None,False,None,Solid Precursor recipe for test3,...,test3,None,NaN,None,None,None,None,None,None,None
5,10k_perovskites,None,None,None,118911,None,None,False,None,Solid Precursor recipe for test5,...,test5,None,NaN,None,None,None,None,None,None,None
6,10k_perovskites,None,None,None,118928,None,None,False,None,Solid Precursor recipe for SP000005,...,SP000005,None,NaN,7787-64-6,C0084566,bismuth (iii) iodide,BiI3,Sigma,2024-10-08,Nano Glovebox
7,10k_perovskites,None,None,None,118929,None,None,False,None,Solid Precursor recipe for SP000006,...,SP000006,None,NaN,7787-58-8,C0169369,bismuth (iii) bromide,BiBr3,Sigma,2024-12-18,Nano Glovebox
8,10k_perovskites,None,None,None,118930,None,None,False,None,Solid Precursor recipe for SP000007,...,SP000007,None,NaN,NaN,NaN,tyrammonium iodide,PEAOHI,Homemade,2025-10-20,6207 Bench
9,10k_perovskites,None,None,None,118931,None,None,False,None,Solid Precursor recipe for SP000008,...,SP000008,None,NaN,NaN,NaN,3-(tert-butoxy)-3-oxopropan-1-aminium iodide,tBuO2CEAI,Homemade,None,6207 Bench


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.organic_salt_sp-id,scientific_metadata.organic_salt_name,scientific_metadata.organic_cation_actual_weight_mg,scientific_metadata.metal_salt_sp-id,scientific_metadata.metal_salt_name,scientific_metadata.metal_cation_actual_weight_mg,scientific_metadata.solvent,scientific_metadata.solvent_volume_ml,scientific_metadata.target_concentration_mol,scientific_metadata.storage_location
0,10k_perovskites,None,None,None,118896,None,None,False,None,Stock Solution recipe for SS00001,...,SP000001,PEAOHI,239.91,NaN,NaN,NaN,DMF,1.006,0.9,None
1,10k_perovskites,None,None,None,118897,None,None,False,None,Stock Solution recipe for SS00002,...,SP000002,PEAOHBr,194.65,NaN,NaN,NaN,DMF,0.992,0.9,None
2,10k_perovskites,None,None,None,118898,None,None,False,None,Stock Solution recipe for SS00003,...,NaN,NaN,NaN,SP000003,BiI3,362.28,DMF,1.024,0.6,None
3,10k_perovskites,None,None,None,118899,None,None,False,None,Stock Solution recipe for SS00004,...,NaN,NaN,NaN,SP000004,BiBr3,273.2,DMF,1.015,0.6,None
4,10k_perovskites,None,None,None,118912,None,None,False,None,Stock Solution recipe for SS00005,...,SP000005,PEAOHI,236.34,NaN,NaN,NaN,DMF,0.991,0.9,None
5,10k_perovskites,None,None,None,118913,None,None,False,None,Stock Solution recipe for SS00006,...,SP000006,tBuO2CEAI,242.77,NaN,NaN,NaN,DMF,0.988,0.9,None
6,10k_perovskites,None,None,None,118914,None,None,False,None,Stock Solution recipe for SS00007,...,SP000007,tBuAI,181.18,NaN,NaN,NaN,DMF,1.001,0.9,None
7,10k_perovskites,None,None,None,118915,None,None,False,None,Stock Solution recipe for SS00008,...,SP000008,Beta-alaHI,196.71,NaN,NaN,NaN,DMF,1.007,0.9,None
8,10k_perovskites,None,None,None,118916,None,None,False,None,Stock Solution recipe for SS00009,...,SP000009,PEAOHBr,191.72,NaN,NaN,NaN,DMF,0.977,0.9,None
9,10k_perovskites,None,None,None,118917,None,None,False,None,Stock Solution recipe for SS00010,...,SP000010,HDABr,139.9,NaN,NaN,NaN,DMF,1.118,0.9,None


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,creation_time,unique_id,file_to_upload,size,owner_user_id,data_format,owner_orcid,measurement,scientific_metadata.sample_positions,scientific_metadata.carrier_image_dataset_id
0,10k_perovskites,None,36,None,121546,None,None,False,photobox,sample well image | TF000177,2026-01-15T19:01:54.440343,0tbg8f8eq1sbs000x1sdqdtpnm,api-uploads/0tbg8f8eq1sbs000x1sdqdtpnm.jpeg,NaN,32,None,0000-0002-8792-9669,sample well image TEST,A1,0tbafxym59xzk000ad2hyj94e8
1,10k_perovskites,None,36,None,121547,/mnt/gcs/api-uploads,f16e1a2718e9a777a669e57ff1517e7087c986f392816a...,False,photobox,sample well image | TF000177,2026-01-15T19:01:54.440343,0tbg8hvnvdtcn000tznjhx1e4r,api-uploads/0tbg8hvnvdtcn000tznjhx1e4r.jpeg,17554.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,A1,0tbafxym59xzk000ad2hyj94e8
2,10k_perovskites,None,36,None,121548,/mnt/gcs/api-uploads,4ec9271a30eee103a182bb682b181686c67f3dbdeff690...,False,photobox,sample well image | TF000178,2026-01-15T19:01:54.440343,0tbg8hzmp5xhv000jgw4x45mcw,api-uploads/0tbg8hzmp5xhv000jgw4x45mcw.jpeg,17675.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,A2,0tbafxym59xzk000ad2hyj94e8
3,10k_perovskites,None,36,None,121549,/mnt/gcs/api-uploads,4dc0845c65041e7fddbb9bd086c8c665a592eba9973927...,False,photobox,sample well image | TF000179,2026-01-15T19:01:54.440343,0tbg8j39c1tpx000sg4nmt32y0,api-uploads/0tbg8j39c1tpx000sg4nmt32y0.jpeg,18026.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,A3,0tbafxym59xzk000ad2hyj94e8
4,10k_perovskites,None,36,None,121550,/mnt/gcs/api-uploads,b75d41d134bf93561e3bd30491b8f32590438308ad91b4...,False,photobox,sample well image | TF000180,2026-01-15T19:01:54.440343,0tbg8j6xqdxeq0009313n6w64r,api-uploads/0tbg8j6xqdxeq0009313n6w64r.jpeg,17185.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,A4,0tbafxym59xzk000ad2hyj94e8
5,10k_perovskites,None,36,None,121551,/mnt/gcs/api-uploads,b1b09597c36085a7f9dfca94ea6e361d91fdc62259f7bc...,False,photobox,sample well image | TF000181,2026-01-15T19:01:54.440343,0tbg8jagz5zn5000d3ctmppqe4,api-uploads/0tbg8jagz5zn5000d3ctmppqe4.jpeg,17647.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,B1,0tbafxym59xzk000ad2hyj94e8
6,10k_perovskites,None,36,None,121552,/mnt/gcs/api-uploads,7c703667b1b0f23ef8369b87446fedf4df23534cfc8973...,False,photobox,sample well image | TF000182,2026-01-15T19:01:54.440343,0tbg8je4bnwhf000vm5ahfwq78,api-uploads/0tbg8je4bnwhf000vm5ahfwq78.jpeg,17615.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,B2,0tbafxym59xzk000ad2hyj94e8
7,10k_perovskites,None,36,None,121553,/mnt/gcs/api-uploads,5c7e50272bddab3dcc2e64938b2a938608631519a81e8e...,False,photobox,sample well image | TF000183,2026-01-15T19:01:54.440343,0tbg8jhrr9rcv000qnmc22t88m,api-uploads/0tbg8jhrr9rcv000qnmc22t88m.jpeg,18134.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,B3,0tbafxym59xzk000ad2hyj94e8
8,10k_perovskites,None,36,None,121554,/mnt/gcs/api-uploads,5156a7fd680877d5a5146b992048ee1a797d28423b8e7b...,False,photobox,sample well image | TF000184,2026-01-15T19:01:54.440343,0tbg8jnd45vhf000579p0y9he4,api-uploads/0tbg8jnd45vhf000579p0y9he4.jpeg,29672.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,B4,0tbafxym59xzk000ad2hyj94e8
9,10k_perovskites,None,36,None,121555,/mnt/gcs/api-uploads,f4f39a1b30ad2d8a91b8958f3552f51cf490a894facfa4...,False,photobox,sample well image | TF000185,2026-01-15T19:01:54.440343,0tbg8js0x1rqf000krg4t3gef4,api-uploads/0tbg8js0x1rqf000krg4t3gef4.jpeg,17373.0,32,jpeg,0000-0002-8792-9669,sample well image TEST,C1,0tbafxym59xzk000ad2hyj94e8


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.heater_position,scientific_metadata.precursor_solution_position,scientific_metadata.heater_sv_temp,scientific_metadata.run_id,scientific_metadata.base_heater_temperature,scientific_metadata.solution_number,scientific_metadata.substrate_scribed,scientific_metadata.substrate_preparation,scientific_metadata.substrate_preparation_timestamp,scientific_metadata.session_name
0,10k_perovskites,None,14,None,119411,None,None,False,SpinBot,Spin Run for TF000001 - 0ta8csyfb1rrs,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10k_perovskites,None,14,None,119412,None,None,False,SpinBot,Spin Run for TF000002 - 0ta8d1mfj1rcb,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10k_perovskites,None,14,None,119413,None,None,False,SpinBot,Spin Run for TF000003 - 0ta8dg19z1s91,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10k_perovskites,None,14,None,119414,None,None,False,SpinBot,Spin Run for TF000003 - 0ta8dhp1axvdv,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10k_perovskites,None,14,None,119415,None,None,False,SpinBot,Spin Run for TF000004 - 0ta8dja7phwnk,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,10k_perovskites,(optional),14,None,121649,None,None,False,SpinBot,Spin Run for TF000572 - 0tbhf9m651tg7,...,D,4.0,160.0,0tbhf9m651tg700059nx79fjyr,160.0,4.0,NA,UV-Ozone,2026-01-21T17:10,(optional)
669,10k_perovskites,(optional),14,None,121650,None,None,False,SpinBot,Spin Run for TF000573 - 0tbhf9m651tg7,...,E,19.0,160.0,0tbhf9m651tg700059nx79fjyr,160.0,19.0,NA,UV-Ozone,2026-01-21T17:10,(optional)
670,10k_perovskites,(optional),14,None,121651,None,None,False,SpinBot,Spin Run for TF000574 - 0tbhf9m651tg7,...,F,20.0,160.0,0tbhf9m651tg700059nx79fjyr,160.0,20.0,NA,UV-Ozone,2026-01-21T17:10,(optional)
671,10k_perovskites,(optional),14,None,121652,None,None,False,SpinBot,Spin Run for TF000575 - 0tbhf9m651tg7,...,G,21.0,160.0,0tbhf9m651tg700059nx79fjyr,160.0,21.0,NA,UV-Ozone,2026-01-21T17:10,(optional)


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.sample_name,scientific_metadata.sample_description,scientific_metadata.notes,scientific_metadata.target_stoichiometry,scientific_metadata.component_a_ss-id,scientific_metadata.component_b_ss-id,scientific_metadata.mixing_ratio,scientific_metadata.target_concentration_(m),scientific_metadata.storage_location,scientific_metadata.ps_autobot_recipe_filename
0,10k_perovskites,None,None,None,118900,None,None,False,None,Precursor Solution recipe for PS000001,...,PS000001,None,Filtered after mixing,(PEAOH)₃BiI₆,SS00001,SS00003,3:1,0.2,FumeHood,None
1,10k_perovskites,None,None,None,118901,None,None,False,None,Precursor Solution recipe for PS000002,...,PS000002,None,Filtered after mixing,(PEAOH)₃Bi₂I₉,SS00001,SS00003,3:2,0.2,FumeHood,None
2,10k_perovskites,None,None,None,118902,None,None,False,None,Precursor Solution recipe for PS000003,...,PS000003,None,Filtered after mixing,(PEAOH)₄Bi₂I₁₀ / (PEAOH)₂BiI₅,SS00001,SS00003,2:1,0.2,FumeHood,None
3,10k_perovskites,None,None,None,118903,None,None,False,None,Precursor Solution recipe for PS000004,...,PS000004,None,Filtered after mixing,(PEAOH)BiI₄,SS00001,SS00003,1:1,0.2,FumeHood,None
4,10k_perovskites,None,None,None,118904,None,None,False,None,Precursor Solution recipe for PS000005,...,PS000005,None,Filtered after mixing,(PEAOH)₃BiBr₆,SS00002,SS00004,3:1,0.2,FumeHood,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,10k_perovskites,None,None,None,121532,None,None,False,None,Precursor Solution recipe for PS000052,...,PS000052,None,filtered in air,(PEAOH)BiBr₄,SS00017,SS00020,1:1,0.2,FumeHood,ManualMixing
61,10k_perovskites,None,None,None,121533,None,None,False,None,Precursor Solution recipe for PS000053,...,PS000053,None,filtered in air,(HDA)₃Bi₂Br₁₂,SS00018,SS00020,3:1,0.2,FumeHood,ManualMixing
62,10k_perovskites,None,None,None,121534,None,None,False,None,Precursor Solution recipe for PS000054,...,PS000054,None,filtered in air,(HDA)₂Bi₂Br₁₀ / (HDA)BiBr₅,SS00018,SS00020,2:1,0.2,FumeHood,ManualMixing
63,10k_perovskites,None,None,None,121535,None,None,False,None,Precursor Solution recipe for PS000055,...,PS000055,None,filtered in air,(HDA)₃Bi₄Br₁₈,SS00018,SS00020,3:2,0.2,FumeHood,ManualMixing


,project_id,session_name,instrument_id,json_link,id,source_folder,sha256_hash_file_to_upload,public,instrument_name,dataset_name,...,scientific_metadata.sample_positions.B3,scientific_metadata.sample_positions.B4,scientific_metadata.sample_positions.C1,scientific_metadata.sample_positions.C2,scientific_metadata.sample_positions.C3,scientific_metadata.sample_positions.C4,scientific_metadata.sample_positions.D1,scientific_metadata.sample_positions.D2,scientific_metadata.sample_positions.D3,scientific_metadata.sample_positions.D4
0,10k_perovskites,None,36,None,120099,C:\Users\lab\Pictures\digiCamControl\10k,1b42bca3cfbc80588b0a038f2a757b36227a9b9065aae4...,False,photobox,2025-12-10_TRAY3_4 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10k_perovskites,None,36,None,120101,C:\Users\lab\Pictures\digiCamControl\10k,0d594c964909d94661b10dac435a0dbe0e9ee1062bf927...,False,photobox,2025-12-17_TRAY5_6 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10k_perovskites,None,36,None,120102,C:\Users\lab\Pictures\digiCamControl\10k,3fef7dc4bcfed2f50c394a1ca0d1592bb9641b1e89104c...,False,photobox,2025-12-17_TRAY7_8 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10k_perovskites,None,36,None,120103,C:\Users\lab\Pictures\digiCamControl\10k,d57a02defabac34083d0a9267ac79751ed28d429fe20e2...,False,photobox,2025-12-17_TRAY9_10 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10k_perovskites,None,36,None,120104,C:\Users\lab\Pictures\digiCamControl\10k,f374cda21932588a5d32d984b624fef3ce70d407f0ad04...,False,photobox,2025-12-17_TRAY11 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10k_perovskites,None,36,None,120105,C:\Users\lab\Pictures\digiCamControl\10k,2b5292a4a19350d0ead159a84d8841a07d6ffaa31eab50...,False,photobox,2025-12-18_TRAY11_12 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10k_perovskites,None,36,None,120106,C:\Users\lab\Pictures\digiCamControl\10k,22b20629bbd2030ee614fe3a10f873cbaa1925430fecc0...,False,photobox,2025-12-18_TRAY13_14 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10k_perovskites,None,36,None,120107,C:\Users\lab\Pictures\digiCamControl\10k,95ff4e7bfd42d3eb4750ae9865b123a7824eed34e47646...,False,photobox,2025-12-18_TRAY15_16 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10k_perovskites,None,36,None,120108,C:\Users\lab\Pictures\digiCamControl\10k,cbe8a64c1151286caf1bfad729eebeb2cced465321069a...,False,photobox,2025-12-18_TRAY17 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10k_perovskites,None,36,None,120387,C:\Users\lab\Pictures\digiCamControl\10k,1b42bca3cfbc80588b0a038f2a757b36227a9b9065aae4...,False,photobox,2025-12-10_TRAY3_4 Carrier Image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
crucible_client.list_datasets(project_id = '10k_perovskites', include_metadata = True)

[{'owner_orcid': '0000-0002-6381-3119',
  'measurement': 'Solid Precursor synthesis',
  'project_id': '10k_perovskites',
  'session_name': None,
  'instrument_id': None,
  'json_link': None,
  'id': 118892,
  'source_folder': None,
  'sha256_hash_file_to_upload': None,
  'public': False,
  'instrument_name': None,
  'dataset_name': 'Solid Precursor recipe for SP000001',
  'creation_time': '2025-10-20',
  'unique_id': '0ta32qq28dxs7000nr397jcpbm',
  'file_to_upload': None,
  'size': None,
  'owner_user_id': 78,
  'data_format': None,
  'scientific_metadata': {'id': 99535,
   'scientific_metadata': {'timestamp': '2025-10-20',
    'operator': 'Declan',
    'sample_name': 'SP000001',
    'sample_description': None,
    'notes': None,
    'cas': 'NaN',
    'rfid': 'NaN',
    'name': 'tyrammonium iodide',
    'abbrev': 'PEAOHI',
    'vendor': 'Homemade',
    'opened_timestamp': '2025-10-20',
    'storage_location': '6207 Bench'},
   'dataset_unique_id': '0ta32qq28dxs7000nr397jcpbm'}},
 {'own